In [ ]:
# File Authorship Information
__author__ = """Matteo Lulli, Luca Biferale, Giacomo Falcucci, 
                Mauro Sbragaglia and Xiaowen Shan"""
__copyright__ = """Copyright 2021, Matteo Lulli, Luca Biferale, 
Giacomo Falcucci, Mauro Sbragaglia and Xiaowen Shan, idea.deploy"""
__license__ = """Permission is hereby granted, free of charge, 
to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software 
without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:
The above copyright notice and this permission notice shall be 
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE."""
__maintainer__ = "Matteo Lulli"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

In [ ]:
# Development cell
%load_ext autoreload
%autoreload 2

# Mesoscale Modelling of the Tolman Length in Multi-component Systems

Authors: Matteo Lulli (1), Luca Biferale (2), Giacomo Falcucci (3), Mauro Sbragaglia (2) and Xiaowen Shan (1)

(1) Department of Mechanics and Aerospace Engineering, Southern University of Science and Technology, Shenzhen, Guangdong 518055, China

(2) Department of Physics \& INFN, University of Rome \"Tor Vergata\", Via della Ricerca Scientifica 1, 00133, Rome, Italy.

(3) Department of Enterprise Engineering \"Mario Lucertini\", University of Rome \"Tor Vergata\", Via del Politecnico 1, 00133 Rome, Italy; John A. Paulson School of Engineering and Applied Physics, *Harvard University*,  33 Oxford Street, 02138 Cambridge, Massachusetts, USA.

**Abstract:**

In this paper we analyze the curvature corrections to the surface tension in the context of the Shan-Chen (SC) multi-component Lattice Boltzmann method (LBM). We demonstrate that the same techniques recently applied in the context of the Shan-Chen multi-phase model can be applied to multi-component mixtures. We implement, as a new application, the calculation of the surface of tension radius $R_s$ through the minimization of the generalized surface tension $\sigma[R]$. In turn we are able to estimate the Tolman length, i.e. the first order coefficient of the curvature expansion of the surface tension $\sigma(R)$, as well as the higher order corrections, i.e. the curvature- and the Gaussian-rigidity coefficients. The SC multi-component model allows to model both fully-symmetric as well as asymmetric interactions among the components. By performing an extensive set of simulations we present a first example of tunable Tolman length in the mesoscopic model, being zero for symmetric interactions and different from zero otherwise. This result paves the way for controlling such interface properties which are paramount in presence of thermal fluctuations. All reported results can be independently reproduced through the ``idea.deploy'' framework available at [https://github.com/lullimat/idea.deploy](https://github.com/lullimat/idea.deploy).

# Reproducibility

This document is intended for those interested readers who want to reproduce the results reported in the paper entitled "Mesoscale Modelling of the Tolman Length in Multi-component Systems". In the present case the computational resources needed should be available in general, although the netire set of simulations takes a time of the order of weeks on NVIDIA P100 cards. This time can considerably be shortened when discarding the simulations of the largest droplest for $L=301$. 

Next development steps will include a class to measure the time required by each cell and output it in a .json file which can be sent to [matteo.lulli@gmail.com](mailto:matteo.lulli@gmail.com) so that average execution times will be availble and organized according to the hardware.

Each subsection can be executed independently and reproduce the results which will be stored locally in the directory 'reproduced-data', so that the data will be generated only once.

Plots can also be generated using the same scripts employed for the figures of the paper. Since there are some issues in executing these scripts in the Jupyter environment we include them as separated files which are called from the cells themselves. **In order to reproduce the plots a working 'latex' installation is necessary to be present on the system.**

This file will be kept updated for new local features and developments in the parent project [**idea.deploy**](https://github.com/lullimat/idea.deploy)

This file is supposed to be pulled from the repository [https://github.com/lullimat/MesoscaleTolmanMC](https://github.com/lullimat/MesoscaleTolmanMC), from within the "papers" directory the idea.deploy project.

**IMPORTANT NOTICE**

**At the present moment this Jupyter notebook has only been tested on the 'devel' branch of the 'idea.deploy' framework. In order to switch branch please go in the root directory of the project from a terminal and type ```git checkout devel```, after which this notebook should be working properly. The repository shall be working on the 'master' branch after the peer-review process is completed.**

**All the scripts in this notebook have been tested with a maximum memory usage of 8GB for the GPU, 64-bits floating point variables and a maximum value for $L$ of at least 213. If you desire to study a different a range of paramters because of hardware limitations feel free to modify the scripts below, especially for the plots.**

# Paper Results
Using the next subsections/cells it is possible to reproduce all the results and plots reported in the paper.

## Simulations: Hardware Selection
In the following cells you are required to make a choice for the hardware to employ for the simulations. At the moment you need to stick to a given choice in order to visualize all the results. A more elastic management will be provided in future releases.

**These are the only cells in the notebook that have a "global" character, i.e. they are needed by the rest of the notebook for the simulations cell to run.**

By executing (shift-enter) the cell below, the hardware present in your system will be listed

In [ ]:
# Display Hardware (click the arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import IdpyHardware

IdpyHardware()

In [ ]:
# Language imports (click on left arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import CUDA_T, OCL_T, idpy_langs_sys

In [ ]:
preferred_lang, preferred_device, preferred_kind = CUDA_T, 0, "gpu"

### One-dimensional interfaces

In [ ]:
# Simulations (click on left arrow to unfold)

import sympy as sp
import numpy as np

from LBM_proxy import CheckCenterOfMassDeltaPConvergence

from TolmanSimulations import TolmanSimulationsMC, TolmanSimulationsFlatMC
from TolmanSimulations import Gcs

from TolmanSimulations import LatticePressureTensorMC, SurfaceOfTensionMC
from idpy.LBM.LBM import XIStencils, FStencils, IndexFromPos
from idpy.LBM.LBM import CheckUConvergence, ComputeCenterOfMass

from idpy.Utils.SimpleTiming import SimpleTiming
_s_timing = SimpleTiming()

obs_list = ['sigma_flat', 'p_n_minus_t_spl', 'p_n', 'p_t',
            'p_n_spl', 'p_t_spl', 'z_fine', 
            'n_field_0', 'n_field_1', 
            'f_stencils', 'psi_syms', 'Gs']

n = sp.Symbol('n')
psis = [sp.exp(-1/n), 1 - sp.exp(-n), n]
psi_codes = {psis[0]: 'exp((NType)(-1./ln))', 
             psis[1]: '1. - exp(-(NType)ln)', 
             n: 'ln'}

from pathlib import Path
reproduced_results = Path("reproduced-results")
if not reproduced_results.is_dir():
    reproduced_results.mkdir()

from collections import defaultdict
analysis_results_flat = defaultdict( # _Gs_i
    lambda: defaultdict(dict) # obs
)

_n_sum_list = [1.92 + 0.12]
_G = 1.5/3

_delta_GAB_list = np.array([0, 0.04, 0.08, 0.16, 0.24, 0.32, 0.40, 0.48, 0.56, 0.64, 0.72, 0.80, 0.88])
_delta_GAB_list = np.sort(np.append(_delta_GAB_list, -_delta_GAB_list[1:]))
print(_delta_GAB_list)


_Gs_list = \
    [
            [[Gcs[psis[0]] * (1 - 0.4 * (1 + _delta_GAB)), _G], 
             [_G, Gcs[psis[0]] * (1 - 0.4)]] for _delta_GAB in _delta_GAB_list 
    ]

_psi_codes = {}
_psi_codes = [[psi_codes[psis[0]], psi_codes[n]], 
              [psi_codes[n], psi_codes[psis[0]]]]

_psi_syms = [[psis[0], n], [n, psis[0]]]

_f_stencils_dict = [[FStencils['D3E4'], FStencils['D3E4']],
                    [FStencils['D3E4'], FStencils['D3E4']]]

_s_timing.Start()
for _n_sum in _n_sum_list:
    for _Gs_i in range(len(_Gs_list)):
        '''
        Preparing analysis results structure
        '''
        for _obs in obs_list:
            analysis_results_flat[_Gs_i][_obs] = []
            
        print("Flat Interface Simulation", _Gs_i)

        simulations_params = {'dim_sizes': (127, 5, 5),
                              'n_components': 2, 
                              'f_stencils': _f_stencils_dict,
                              'n_sum': _n_sum,
                              'psi_syms' : _psi_syms,
                              'psi_codes': _psi_codes,
                              'SC_Gs': _Gs_list[_Gs_i], 
                              'taus': [1., 1.],                       
                              'lang': preferred_lang,
                              'cl_kind': preferred_kind, 
                              'device': preferred_device, 
                              'data_dir': reproduced_results}
        
        for _param in ['SC_Gs']:
            print(simulations_params[_param])

        _ts_test = TolmanSimulationsFlatMC(**simulations_params)

        _did_run = _ts_test.Simulate()
        
        _swap_out = _ts_test.GetFlatSigma()
        for _obs in _swap_out:
            analysis_results_flat[_Gs_i][_obs] += [_swap_out[_obs]]

        _ts_test.End()

        print()
        print()
        
        
'''
Creating numpy arrays for the data analysis
'''
for _Gs_i in analysis_results_flat:
    for _obs in obs_list:
        analysis_results_flat[_Gs_i][_obs] = \
            np.array(analysis_results_flat[_Gs_i][_obs])
        
_s_timing.End()
_s_timing.PrintElapsedTime()

### Droplets

In [ ]:
# Cell printing the estimated allocated memory size in (MB) for the simulations (click on left arrow to unfold)
def MemoryEstimateMB(_L, _dim = 3, _Q = 19):
    _V = _L ** _dim
    _U_V = _dim * _V
    _Pop_V = _Q * _V
    _Psi_V, _N_V = _V, _V
    _memory = _Psi_V * 2 + _N_V * 3 + _U_V * 3 + _Pop_V * 3
    _memory *= 8
    return _memory / 2 ** 20

print("Estimated simulation sizes in MB:")
_Ls_unique = [25, 31, 37, 41, 51, 57, 65, 97, 127, 151, 213, 301]
for _L in _Ls_unique:
    print("L:", _L, "Size:", "{:.1f}".format(MemoryEstimateMB(_L)), "(MB)")

In the cell below we set the range memory usage (i.e. ```_M_min, _M_max```) and the number of values of the asymmetry paramter $\delta G_{AB}$ (i.e. ```_Gs_slice```). For ```_Gs_slice``` one should use the ```slice``` class. 

The full set of simulations can be obtained by setting ```_M_min, _M_max = 0, 2**14``` and ```_Gs_slice = slice(0, -1, 1)```

In [ ]:
_M_min, _M_max = 0, 2**13
_Gs_slice = slice(0, 1, 1)

Below we list the time needed for the set of simulations involving only one value of $\delta G_{AB}$ (e.g. ```_Gs_slice = slice(0, 1, 1)```) with the system size limited to $L \leq 213$ (i.e. ```_M_min, _M_max = 0, 2**13```)

*Flat interfaces already simulated in Sec. 3.1.1*

**0 d,  4 h,  49 m,  19 s, CUDA, Tesla P100-PCIE-16GB, Intel(R) Xeon(R) CPU E5-2620 v4 @ 2.10GHz**

In order to create the plots in Sec. 3.2, 3.3 and 3.4 it is necessaryo to execute the cell below first.

In [ ]:
# Simulations (click the arrow on the left to unfold)

import sympy as sp
import numpy as np

from LBM_proxy import CheckCenterOfMassDeltaPConvergence

from TolmanSimulations import TolmanSimulationsMC, TolmanSimulationsFlatMC
from TolmanSimulations import Gcs

from TolmanSimulations import LatticePressureTensorMC, SurfaceOfTensionMC
from idpy.LBM.LBM import XIStencils, FStencils, IndexFromPos
from idpy.LBM.LBM import CheckUConvergence, ComputeCenterOfMass

from idpy.Utils.SimpleTiming import SimpleTiming
_s_timing = SimpleTiming()

obs_list = ['delta_p', 'radial_pt_spl', 'radial_pn_spl', 'delta_p_st',
            'sigma_4.217', 'Rs_4.217', 'st_spl_4.217', 'r_fine_4.217', 'L']

n = sp.Symbol('n')
psis = [sp.exp(-1/n), 1 - sp.exp(-n), n]
psi_codes = {psis[0]: 'exp((NType)(-1./ln))', 
             psis[1]: '1. - exp(-(NType)ln)', 
             n: 'ln'}

from pathlib import Path
reproduced_results = Path("reproduced-results")
if not reproduced_results.is_dir():
    reproduced_results.mkdir()

from collections import defaultdict
analysis_results = defaultdict( # _Gs_i
    lambda: defaultdict( # 'A'/'B'
        lambda: defaultdict(dict) # obs
    )
)

# 301
_Ls_list = [25, 31, 37, 41, 51, 57, 65, 97, 127, 151, 213, 301]

_Ls_list_swap = []
for _L in _Ls_list:
    if MemoryEstimateMB(_L) >= _M_min and MemoryEstimateMB(_L) < _M_max:
        _Ls_list_swap += [_L]
_Ls_list = _Ls_list_swap.copy()
print(_Ls_list)

_n_sum_list = [1.92 + 0.12]
_G = 1.5/3


_delta_GAB_list = np.array([0, 0.04, 0.08, 0.16, 0.24, 0.32, 0.40, 0.48, 0.56, 0.64, 0.72, 0.80, 0.88])
_delta_GAB_list = np.sort(np.append(_delta_GAB_list, -_delta_GAB_list[1:]))
print(_delta_GAB_list)


_Gs_list = \
    [
            [[Gcs[psis[0]] * (1 - 0.4 * (1 + _delta_GAB)), _G], 
             [_G, Gcs[psis[0]] * (1 - 0.4)]] for _delta_GAB in _delta_GAB_list 
    ]

_psi_codes = {}
_psi_codes = [[psi_codes[psis[0]], psi_codes[n]], 
              [psi_codes[n], psi_codes[psis[0]]]]

_psi_syms = [[psis[0], n], [n, psis[0]]]

_f_stencils_dict = [[FStencils['D3E4'], FStencils['D3E4']],
                    [FStencils['D3E4'], FStencils['D3E4']]]


_s_timing.Start()

for _n_sum in _n_sum_list:
    for _Gs_i in range(len(_Gs_list))[_Gs_slice]:
        for _type in ['A', 'B']:
            '''
            Preparing analysis results structure
            '''
            for _obs in obs_list:
                analysis_results[_Gs_i][_type][_obs] = []

            for _L in _Ls_list:
                simulations_params = {'dim_sizes': (_L, _L, _L),
                                      'n_components': 2, 
                                      'f_stencils': _f_stencils_dict,
                                      'n_sum': _n_sum,
                                      'psi_syms' : _psi_syms,
                                      'psi_codes': _psi_codes,
                                      'R': _L/4, 'type': _type,
                                      'SC_Gs': _Gs_list[_Gs_i], 
                                      'taus': [1., 1.],                       
                                      'lang': preferred_lang,
                                      'cl_kind': preferred_kind, 
                                      'device': preferred_device, 
                                      'data_dir': reproduced_results, 
                                      'flat_sim_cache': True}

                _ts_test = TolmanSimulationsMC(**simulations_params)

                _did_run = _ts_test.Simulate(override_flag = False)
                
                if _did_run not in ['burst', 'center']:
                    print(_Gs_i, _type)
                    '''
                    System size
                    '''
                    analysis_results[_Gs_i][_type]['L'] += [_L]
                    '''
                    Delta p
                    '''
                    _swap_out = _ts_test.GetDataDeltaP()
                    for obs in _swap_out:
                        analysis_results[_Gs_i][_type][obs] += [_swap_out[obs]]
                    '''
                    Surface Of Tension Radius
                    '''
                    SR = SurfaceOfTensionMC(**_ts_test.GetDataSurfaceOfTension())
                    _swap_out = SR.GetSurfaceTension()
                    for obs in _swap_out:
                        analysis_results[_Gs_i][_type][obs] += [_swap_out[obs]]

                _ts_test.End()
                print()
            '''
            End _type loop
            '''
            print()
        '''
        End _Gs loop
        '''
        
        
'''
Creating numpy arrays for the data analysis
'''
for _Gs_i in analysis_results:
    for _type in analysis_results[_Gs_i]:
        for _obs in obs_list:
            analysis_results[_Gs_i][_type][_obs] = \
                np.array(analysis_results[_Gs_i][_type][_obs])
            
            
_s_timing.End()
_s_timing.PrintElapsedTime()

## Figure 2

In [ ]:
# Figure 2 script (click the arrow on the left to unfold)

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
rc('font',**{'family':'STIXGeneral'})
rc('mathtext', **{'fontset': 'stix'})
rc('text', usetex=True)

rcParams['text.latex.preamble']=[r"\usepackage{amsmath, sourcesanspro}"]

_fs, _ms_small, _ms_large, _ms_large_c = 20, 9, 12, 20

_l_fs = 17

rc('legend', fontsize = _l_fs)

from pathlib import Path
reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()
    
_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])

_Gs_len = len(_Gs_list[_Gs_slice])

_sym_i = _Gs_len//2
print("Symmetric index:", _sym_i)

_Gs_i_range = range(_Gs_len)
#_Gs_i_range = [1, 8, 15]
_Gs_AA_list = np.array([abs(_Gs_list[_][0][0]) for _ in _Gs_i_range])
##_delta_Gs = np.array([1 - (1 - _Gs_list[_Gs_i][0][0] / Gcs[psis[0]]) / 0.4 for _Gs_i in _Gs_i_range])
_delta_Gs = np.array([(1 - _Gs_list[_Gs_i][0][0] / Gcs[psis[0]]) / 0.4 - 1 for _Gs_i in _Gs_i_range])

_Gs_col_norm = matplotlib.colors.LogNorm(vmin = np.amin(_Gs_AA_list), 
                                         vmax = np.amax(_Gs_AA_list))

fig = plt.figure(figsize = (8.3, 13))

######### Generalized surface tension
_panel_str = '$(a)$'
_ax_gen_sigma = plt.subplot2grid((2, 1), (0, 0), colspan = 1, rowspan = 1)
_x_range = [0.56,2]
if True:
    _ii = 0
    for _i in _Gs_i_range:
        _color = _cmap(_Gs_col_norm(abs(_Gs_list[_i][0][0])))

        for _type in ['A', 'B']:
            _every_L = 5
            for _k in range(len(analysis_results[_i][_type]['r_fine_4.217']))[::_every_L]:
                _swap_Rs_4p217 = analysis_results[_i][_type]['Rs_4.217'][_k]
                _swap_sigma_4p217 = analysis_results[_i][_type]['sigma_4.217'][_k]

                _swap_r = \
                    np.linspace(_swap_Rs_4p217 * _x_range[0], 
                                _swap_Rs_4p217 * _x_range[1], 149) + \
                    _ii * _swap_Rs_4p217 * (_x_range[1] - _x_range[0]) / 149

                _swap_st = analysis_results[_i][_type]['st_spl_4.217'][_k](_swap_r)

                _freq = 128 + 64 + 16 + 16
                _ax_gen_sigma.plot(_swap_r[0::_freq]/_swap_Rs_4p217, 
                                   _swap_st[0::_freq]/_swap_sigma_4p217, 
                                   'o', 
                                   color = _color, markersize = _ms_large,
                                   fillstyle = 'none' if _type == 'B' else 'full')
                ##print(_ii)
                _ii += 1

    _r_u = np.linspace(0.2, 3, 2 ** 8)
    _ax_gen_sigma.plot(_r_u, 1 + ((1 - 1/_r_u) ** 2) * (1 + 2 * _r_u) / 3, 
                       label = 'Integrated generalized Laplace law: \
                       $\\frac{\\sigma\\left[R\\right]}{\\sigma_{s}}=\\frac{1}{3}\\left(\\frac{R_{s}}{R}\\right)^{2}+\\frac{2}{3}\\frac{R}{R_{s}}$', color = 'orange', 
                       linestyle = '-.', linewidth = 4)

    _ax_gen_sigma.axhline(y = 1, color = 'black', linestyle = '-.', linewidth = 0.85)
    _ax_gen_sigma.axvline(x = 1, ymin = (1 - 0.85)/(2 - 0.85), ymax = 1, 
                          color = 'black', linestyle = '-.', linewidth = 0.85)

    _ax_gen_sigma.text(0.015, 0.925, _panel_str, transform = _ax_gen_sigma.transAxes, fontsize=_fs)

    ##_ax_gen_sigma.plot(_r_fine/_Rs, 1 / ((_r_fine/_Rs) ** 2) / 3 + 2 * (_r_fine/_Rs) / 3)

    _ax_gen_sigma.set_xlabel('$R/R_s$', fontsize = _fs)
    _ax_gen_sigma.set_ylabel('$\sigma[R]/\sigma_s$', fontsize = _fs)

    _ax_gen_sigma.set_xlim(_x_range)
    _ax_gen_sigma.set_ylim([0.85,1.6])

    for tick in _ax_gen_sigma.xaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    for tick in _ax_gen_sigma.yaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    lgnd_points = _ax_gen_sigma.legend(frameon = False, loc = 'lower right')

######### Laplace Law
_panel_str = '$(b)$'
_ax_laplace = plt.subplot2grid((2, 1), (1, 0), colspan = 1, rowspan = 1)


_symbols_dict = {0: 'o', _sym_i: 's', _Gs_len - 1: '^'}
#for _i in [15, 1, 8]:
for _i in [0, _sym_i, _Gs_len - 1]:
    _color = _cmap(_Gs_col_norm(abs(_Gs_list[_i][0][0])))

    _rm1_A = 1 / analysis_results[_i]['A']['Rs_4.217']
    plt.plot(_rm1_A, 
             analysis_results[_i]['A']['delta_p_st'], _symbols_dict[_i], 
             color = _color, markersize = _ms_small,
             fillstyle = 'full', label = '$\delta G_{AB} = ' + ("%.02f" % _delta_Gs[_i]) + '$')

    _rm1_B = -1 / analysis_results[_i]['B']['Rs_4.217']
    plt.plot(_rm1_B, 
             analysis_results[_i]['B']['delta_p_st'], _symbols_dict[_i], 
             color = _color, markersize = _ms_small,
             fillstyle = 'none')
    
    _sigma_flat = analysis_results_flat[_i]['sigma_flat'][0]
    _rm1_fine_A = np.linspace(0, np.amax(_rm1_A), 2 ** 7)
    _ax_laplace.plot(_rm1_fine_A, 
                     + 2 * _sigma_flat * _rm1_fine_A, 
                     '--', color = _color)

    _rm1_fine_B = np.linspace(np.amin(_rm1_B), 0, 2 ** 7)
    _ax_laplace.plot(_rm1_fine_B, 
                     - 2 * _sigma_flat * _rm1_fine_B, 
                     '--', color = _color)
    
_ax_laplace.set_xlabel('$R_s^{-1}$', fontsize = _fs)
_ax_laplace.set_ylabel('$\Delta P$', fontsize = _fs)

_ax_laplace.axvline(x = 0, color = 'black', linestyle = '-.', linewidth = 0.5)
_ax_laplace.text(0.075, 0.125, 'B-droplets', transform = _ax_laplace.transAxes, fontsize = _fs)
_ax_laplace.text(0.725, 0.125, 'A-droplets', transform = _ax_laplace.transAxes, fontsize = _fs)
_ax_laplace.text(0.015, 0.925, _panel_str, transform = _ax_laplace.transAxes, fontsize=_fs)


for tick in _ax_laplace.xaxis.get_major_ticks():
    tick.label.set_fontsize(_fs)

for tick in _ax_laplace.yaxis.get_major_ticks():
    tick.label.set_fontsize(_fs)

lgnd_points = _ax_laplace.legend(frameon = True, loc = 'upper center', 
                                 bbox_transform = _ax_laplace.transAxes, 
                                 bbox_to_anchor = (0.3,1))

_ax_laplace.set_ylim([0., 0.085])

#for _handle in lgnd_points.legendHandles:
#    _handle._legmarker.set_color('black')

plt.savefig(reproduced_figures / 'figure_gensigma_laplace.pdf', bbox_inches = 'tight', dpi = 200)

## Figure 3

### Analysis Block

In [ ]:
# Analysis for Figures 3 & 4 (click the arrow on the left to unfold)

from scipy import interpolate, optimize

_fit_obs_list = ['first_order', 'second_order', 'sigma_0']

_delta_fit_res = defaultdict(dict)
for _obs in _fit_obs_list:
    _delta_fit_res[_obs] = []
    
_Gs_len = len(_Gs_list[_Gs_slice])
_Gs_i_range = range(_Gs_len)
    
_k = 0
for _i in _Gs_i_range:
    '''
    The aim is to interpolate the data first and then to fit them
    - from the interpolation we get a first estimate of the flat surface tension and of \delta
    - we use the results of the interpolation to inform the fits
    - We should use a third order polynomial for the fits this time (compare with lit.)
    '''
    _Rsm1_A = 1 / analysis_results[_i]['A']['Rs_4.217']
    _Rsm1_B = -1 / analysis_results[_i]['B']['Rs_4.217']
    _Rs_A = analysis_results[_i]['A']['Rs_4.217']
    _Rs_B = analysis_results[_i]['B']['Rs_4.217']
    _delta_p_sigma_s_A = _Rs_A * analysis_results[_i]['A']['delta_p_st'] / 2
    _delta_p_sigma_s_B = _Rs_B * analysis_results[_i]['B']['delta_p_st'] / 2

    _swap_sigma_0 = analysis_results_flat[_i]['sigma_flat']
    if True:
        _rm1_data = np.append(_Rsm1_B, np.append(_Rsm1_A, [0]))
        _delta_p_sigma_s_data = np.append(_delta_p_sigma_s_B, np.append(_delta_p_sigma_s_A, [_swap_sigma_0]))
    else:
        _rm1_data = np.append(_Rsm1_B, _Rsm1_A)
        _delta_p_sigma_s_data = np.append(_delta_p_sigma_s_B, _delta_p_sigma_s_A)

    '''
    Interpolation
    '''
    _sorted_args = np.argsort(_rm1_data)
    _rm1_data = _rm1_data[_sorted_args]
    _delta_p_sigma_s_data = _delta_p_sigma_s_data[_sorted_args]

    _delta_p_sigma_s_spl = interpolate.UnivariateSpline(_rm1_data, 
                                                        _delta_p_sigma_s_data, 
                                                        k = 5, s = 0)
    _sigma_0_data = _delta_p_sigma_s_spl(0)
    print(_swap_sigma_0, _sigma_0_data)
    
    _swap_d0 = _delta_p_sigma_s_spl(0)
    _swap_d1 = _delta_p_sigma_s_spl.derivative(1)(0)
    _swap_d2 = _delta_p_sigma_s_spl.derivative(2)(0)
    _swap_d3 = _delta_p_sigma_s_spl.derivative(3)(0)
    _swap_d4 = _delta_p_sigma_s_spl.derivative(4)(0)

    '''
    Linear Fits
    '''
    _fit_range = [-0.075, 0.075]
    #_fit_range = [-0.09, 0.09]
    _fit_cut_index = np.where((_rm1_data > _fit_range[0]) & (_rm1_data < _fit_range[1]))

    _fit_range_shrink = 1 - 0.1
    _rm1_fine_fit = np.linspace(_fit_range[0] * _fit_range_shrink, 
                                _fit_range[1] * _fit_range_shrink, 
                                2 ** 10)

    _fit_range_enlarge = 2.2
    _rm1_fine_lin = np.linspace(_fit_range[0] * _fit_range_enlarge, 
                                _fit_range[1] * _fit_range_enlarge, 
                                2 ** 10)


    '''
    Fourth-order fits yield the best result
    '''
    def _fourth_func(x, a, b, c, d, e):
        return a + b * x + c * (x ** 2) + d * (x ** 3) + e * (x ** 4)

    def _quad_func(x, a, b, c):
        return a + b * x + c * (x ** 2)

    _fourth_fit, _fourth_cov = \
        optimize.curve_fit(_fourth_func, 
                           _rm1_data[_fit_cut_index], 
                           _delta_p_sigma_s_data[_fit_cut_index], 
                           p0 = (_swap_d0, _swap_d1, _swap_d2, _swap_d3, _swap_d4))

    _quad_fit, _quad_cov = \
        optimize.curve_fit(_quad_func, 
                           _rm1_data[_fit_cut_index], 
                           _delta_p_sigma_s_data[_fit_cut_index], 
                           p0 = (_swap_d0, _swap_d1, _swap_d2))

    _fourth_fit_lambda = lambda x : _fourth_func(x, _fourth_fit[0], _fourth_fit[1], 
                                                 _fourth_fit[2], _fourth_fit[3], 
                                                 _fourth_fit[4])
    _quad_fit_lambda = lambda x : _quad_func(x, _fourth_fit[0], _fourth_fit[1], _fourth_fit[2])
    
    #_delta_fit_res['sigma_0'] += [_sigma_0_data]
    _delta_fit_res['sigma_0'] += [_quad_fit[0]]
    
    _delta_fit_res['first_order'] += [_quad_fit[1]/_quad_fit[0]]
    _delta_fit_res['second_order'] += [_quad_fit[2]/_quad_fit[0]]

    print("Tolman length:", -_delta_fit_res['first_order'][-1] / 2)
    print()
        
_delta_fit_res['first_order'] = np.array(_delta_fit_res['first_order'])
_delta_fit_res['second_order'] = np.array(_delta_fit_res['second_order'])

### Plot

In [ ]:
# Figure 3 script (click the arrow on the left to unfold)

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
rc('font',**{'family':'STIXGeneral'})
rc('mathtext', **{'fontset': 'stix'})
rc('text', usetex=True)
rcParams['text.latex.preamble']=[r"\usepackage{amsmath, sourcesanspro}"]

from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

_fs, _ms_small, _ms_large, _ms_large_c = 20, 9, 12, 20
_fs_small = 12
_l_fs = 17
_line_w = 2

rc('legend', fontsize = _l_fs)

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()
    
_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])


_Gs_len = len(_Gs_list[_Gs_slice])
_Gs_i_range = range(_Gs_len)
_sym_i = _Gs_len//2

_Gs_AA_list = np.array([abs(_Gs_list[_][0][0]) for _ in _Gs_i_range])
##_delta_Gs = np.array([1 - (1 - _Gs_list[_Gs_i][0][0] / Gcs[psis[0]]) / 0.4 for _Gs_i in _Gs_i_range])
_delta_Gs = np.array([(1 - _Gs_list[_Gs_i][0][0] / Gcs[psis[0]]) / 0.4 - 1 for _Gs_i in _Gs_i_range])

_Gs_col_norm = matplotlib.colors.LogNorm(vmin = np.amin(_Gs_AA_list), 
                                         vmax = np.amax(_Gs_AA_list))

fig = plt.figure(figsize = (8.3, 11.5))
_ax_sigma_0 = plt.subplot2grid((2, 1), (0, 0), colspan = 1, rowspan = 1)
_panel_str = '$(a)$'

if True:
    _ax_sigma_0.plot(_delta_Gs, _delta_fit_res['sigma_0'], 'o',
                     color = 'red', markersize = _ms_small, label = '$R_s \\to \infty$')


    _delta_flat_array = np.array([analysis_results_flat[_i]['sigma_flat'][0] for _i in _Gs_i_range])
    
    '''
    We can plot the result of the fit with a quartic function only
    when the number of points is larger than 5
    '''
    if len(_delta_flat_array) > 5:
        def _fourth_func(x, a, b, c, d, e):
            return a + b * x + c * (x ** 2) + d * (x ** 3) + e * (x ** 4)

        _fourth_fit, _fourth_cov = \
            optimize.curve_fit(_fourth_func, 
                               _delta_Gs, 
                               _delta_flat_array)

        _lambda_fourth_fit = lambda x: _fourth_func(x, _fourth_fit[0], 
                                                    _fourth_fit[1], _fourth_fit[2], 
                                                    _fourth_fit[3], _fourth_fit[4])

        if False:
            _ax_sigma_0.plot(_delta_Gs, _delta_flat_array, 'o', fillstyle = 'none',
                             color = 'red', markersize = _ms_large_c, label = 'Flat Interface')
        if True:
            _fine_delta_Gs = np.linspace(np.amin(_delta_Gs), np.amax(_delta_Gs), 2 ** 8)
            _ax_sigma_0.plot(_fine_delta_Gs, _lambda_fourth_fit(_fine_delta_Gs), '--',
                             color = 'red', markersize = _ms_large_c, label = 'Flat Interface')


    _ax_sigma_0.set_xlabel('$\delta G_{{AB}}$', fontsize = _fs)
    _ax_sigma_0.set_ylabel("$\\sigma_0$", fontsize = _fs)

    ####### INSET RELATIVE VARIATION WITH RESPECT TO \sigma_0
    _ax_sigma_0_bis = inset_axes(_ax_sigma_0, width='40%', height='40%', loc = 'upper center')

    _ax_sigma_0_bis.plot(_delta_Gs, _delta_fit_res['sigma_0']/np.amax(_delta_fit_res['sigma_0']), 
                         'o', color = 'blue')
    _ax_sigma_0_bis.set_ylabel("$\\sigma_0/\\sigma_{\small{\mbox{max}}}$", fontsize = _fs_small)
    _ax_sigma_0_bis.set_xlim([-0.99, 0.99])
    _ax_sigma_0_bis.set_ylim([0.79, 1.01])

    for tick in _ax_sigma_0.xaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    for tick in _ax_sigma_0.yaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    lgnd_points = _ax_sigma_0.legend(frameon = True, loc = 'lower right')

    _ax_sigma_0.set_ylim([0.1325, 0.185])
    _ax_sigma_0.set_xlim([-0.99, 0.99])
    _ax_sigma_0.text(0.015, 0.035, _panel_str, 
                     transform = _ax_sigma_0.transAxes, fontsize=_fs)
    
#################################
    
_ax_sigma_corr = plt.subplot2grid((2, 1), (1, 0), colspan = 1, rowspan = 1)
_panel_str = '$(b)$'
if True:
    _symbols_dict = {0: 'o', _sym_i: 's', _Gs_len - 1: '^'}
    _lines_widths = {0: _line_w, _sym_i: _line_w * 2, _Gs_len - 1: _line_w}

    for _i in [0, _sym_i, _Gs_len - 1]:
        _Rsm1_A = 1 / analysis_results[_i]['A']['Rs_4.217']
        _Rsm1_B = -1 / analysis_results[_i]['B']['Rs_4.217']
        _Rs_A = analysis_results[_i]['A']['Rs_4.217']
        _Rs_B = analysis_results[_i]['B']['Rs_4.217']
        _delta_p_sigma_s_A = _Rs_A * analysis_results[_i]['A']['delta_p_st'] / 2
        _delta_p_sigma_s_B = _Rs_B * analysis_results[_i]['B']['delta_p_st'] / 2

        _fit_range = [-0.075, 0.075]
        _fit_range_enlarge = 1.9
        _rm1_fine_lin = np.linspace(_fit_range[0] * _fit_range_enlarge, 
                                    _fit_range[1] * _fit_range_enlarge, 
                                    2 ** 10)

        _rm1_fine_fit = np.linspace(_fit_range[0] * _fit_range_enlarge * 0.7, 
                                    _fit_range[1] * _fit_range_enlarge * 0.7, 
                                    2 ** 10)

        def _quad_func(x, b, c):
            return 1 + b * x + c * (x ** 2)

        _quad_fit_lambda = lambda x : \
            _quad_func(x, _delta_fit_res['first_order'][_i], 
                       _delta_fit_res['second_order'][_i])

        _sigma_0_data = _delta_fit_res['sigma_0'][_i]

        _color = _cmap(_Gs_col_norm(abs(_Gs_list[_i][0][0])))
        _ax_sigma_corr.plot(_Rsm1_A, _delta_p_sigma_s_A / _sigma_0_data, 
                            _symbols_dict[_i], color = _color, markersize = _ms_small, 
                            label = '$\delta G_{{AB}} = ' + ("%.02f" % _delta_Gs[_i]) + '$')


        _ax_sigma_corr.plot(_Rsm1_B, _delta_p_sigma_s_B / _sigma_0_data, 
                            _symbols_dict[_i], 
                            markersize = _ms_small, fillstyle = 'none', color = _color)

        _ax_sigma_corr.plot(_rm1_fine_lin, 1 + _delta_fit_res['first_order'][_i] * (_rm1_fine_lin), 
                            color = _color, linestyle = '-.', linewidth = _lines_widths[_i])

        _ax_sigma_corr.plot(_rm1_fine_fit, _quad_fit_lambda(_rm1_fine_fit), 
                            color = _color, linestyle = '--', linewidth = _lines_widths[_i])

    _ax_sigma_corr.plot([0], [1], 's', color = 'black', markersize = _ms_small)

    _ax_sigma_corr.axvline(x = 0, color = 'black', linestyle = '-.', linewidth = 0.5)
    _ax_sigma_corr.axhline(y = 1, color = 'black', linestyle = '-.', linewidth = 0.5)

    _ax_sigma_corr.text(0.015, 0.035, _panel_str, 
                        transform = _ax_sigma_corr.transAxes, fontsize=_fs)
    _ax_sigma_corr.set_xlabel('$R_s^{-1}$', fontsize=_fs)
    _ax_sigma_corr.set_ylabel('$\Delta P \cdot R_s / 2 \sigma_0$', fontsize=_fs)

    for tick in _ax_sigma_corr.xaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    for tick in _ax_sigma_corr.yaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    lgnd_points = _ax_sigma_corr.legend(frameon = True, loc = 'upper center', 
                                        bbox_transform = _ax_laplace.transAxes, 
                                        bbox_to_anchor = (0.29,0.71))
    

plt.savefig(reproduced_figures / 'figure_sigma0_sigmaR.pdf', bbox_inches = 'tight', dpi = 200)

## Figure 4

In [ ]:
# Figure 4 script (click the arrow on the left to unfold)

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
rc('font',**{'family':'STIXGeneral'})
rc('mathtext', **{'fontset': 'stix'})
rc('text', usetex=True)
rcParams['text.latex.preamble']=[r"\usepackage{amsmath, sourcesanspro}"]

_fs, _ms_small, _ms_large, _ms_large_c = 20, 9, 12, 20
_l_fs = 17
_line_w = 2

rc('legend', fontsize = _l_fs)

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()
    
_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])


_Gs_len = len(_Gs_list[_Gs_slice])
_Gs_i_range = range(_Gs_len)
_sym_i = _Gs_len//2

_Gs_AA_list = np.array([abs(_Gs_list[_][0][0]) for _ in _Gs_i_range])
##_delta_Gs = np.array([1 - (1 - _Gs_list[_Gs_i][0][0] / Gcs[psis[0]]) / 0.4 for _Gs_i in _Gs_i_range])
_delta_Gs = np.array([(1 - _Gs_list[_Gs_i][0][0] / Gcs[psis[0]]) / 0.4 - 1 for _Gs_i in _Gs_i_range])

_Gs_col_norm = matplotlib.colors.LogNorm(vmin = np.amin(_Gs_AA_list), 
                                         vmax = np.amax(_Gs_AA_list))

fig = plt.figure(figsize = (8.3, 10.5))
_ax_delta = plt.subplot2grid((2, 1), (0, 0), colspan = 1, rowspan = 1)
_panel_str = '$(a)$'

if True:
    _ax_delta.plot(_delta_Gs, -_delta_fit_res['first_order'] / 2, 'o', 
                   color = 'darkgreen', markersize = _ms_small)
    _ax_delta.set_xlabel('$\delta G_{{AB}}$', fontsize = _fs)
    _ax_delta.set_ylabel('$\delta$', fontsize = _fs)

    _ax_delta.text(0.015, 0.925, _panel_str, transform = _ax_delta.transAxes, fontsize=_fs)

    for tick in _ax_delta.xaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    for tick in _ax_delta.yaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    _ax_delta.axhline(y = 0, color = 'black', linestyle = '-.', linewidth = 0.5)
    _ax_delta.axvline(x = 0, color = 'black', linestyle = '-.', linewidth = 0.5)
    _ax_delta.set_xlim([-0.99, 0.99])
    _ax_delta.set_ylim([-0.69, 0.39])
    
############################################################

_ax_curv_coeffs = plt.subplot2grid((2, 1), (1, 0), colspan = 1, rowspan = 1)
_panel_str = '$(b)$'

if True:
    _ax_curv_coeffs.plot(_delta_Gs, _delta_fit_res['second_order'], 'o', 
                         color = 'orange', markersize = _ms_small)
    _ax_curv_coeffs.set_xlabel('$\delta G_{{AB}}$', fontsize = _fs)
    _ax_curv_coeffs.set_ylabel("$2\\bar{k} + k$", fontsize = _fs)
    
    _ax_curv_coeffs.text(0.015, 0.925, _panel_str, transform = _ax_curv_coeffs.transAxes, fontsize=_fs)    

    for tick in _ax_curv_coeffs.xaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    for tick in _ax_curv_coeffs.yaxis.get_major_ticks():
        tick.label.set_fontsize(_fs)

    _ax_curv_coeffs.axvline(x = 0, color = 'black', linestyle = '-.', linewidth = 0.5)
    _ax_curv_coeffs.set_xlim([-0.99, 0.99])
    _ax_curv_coeffs.set_ylim([23.9, 38.1])

plt.savefig(reproduced_figures / 'figure_curv_coeffs.pdf', bbox_inches = 'tight', dpi = 200)